In [12]:
import json
import re

def extract_product_ids(messages, prompt_response):
    # Extract the JSON string from the prompt_response

 
    json_match = re.search(r'message=ChatCompletionMessage\(content=\'(.*?)\'\)', prompt_response)
    if not json_match:
        raise ValueError("No valid JSON found in prompt_response")

    # Extract the JSON part
    json_str = json_match.group(1).replace("'", '"')
    
    # Fix the single quotes to double quotes for valid JSON
    json_str = json_str.replace("'", '"')
    
    # Parse the valid JSON string
    response_dict = json.loads(json_str)
   
    product_ids = response_dict['product_ids']
    
    # Extract the product names from the HTML formatted message
    product_names = re.findall(r'<li>(.*?)<\/li>', response_dict['message'])

    # Extract products from messages
    message_content = messages[-1]['content']  # The last message is the function call content
    products_data = json.loads(re.search(r'\[(.*?)\]', message_content).group(0))

    # Create a mapping from product name to product ID
    product_id_map = {product['product_name']: product['product_id'] for product in products_data}

    # Prepare the output
    matched_product_names = []
    matched_product_ids = []

    for name in product_names:
        if name in product_id_map:
            matched_product_names.append(name)
            matched_product_ids.append(product_id_map[name])

    # Create the final response
    final_response = {
        "message": f"I found the following chocolate products for you:<br/><ul>" + ''.join(f"<li>{name}</li>" for name in matched_product_names) + "</ul>",
        "product_ids": matched_product_ids
    }
    
    return final_response

# Sample inputs
messages = [
    {'role': 'system', 'content': '[Style]\nWhenever you are listing things, use HTML tags...'},
    {'role': 'assistant', 'content': "Hello, I am here to help with your shopping."},
    {'role': 'user', 'content': 'Filter by chocolate\n'},
    {'role': 'function', 'content': '@agent-action: You found the restaurant page(s) for [{"search_type":"product","product_id":27,"product_name":"KitKat White Chocolate 41.5g","product_quantity":"41.5g","product_brand":"Nestle","product_price":1.9,"product_category":"Confectionery"},{"search_type":"product","product_id":30,"product_name":"Nestle Milkybar White Chocolate 25g","product_quantity":"25g","product_brand":"Nestle","product_price":1.5,"product_category":"Confectionery"},{"search_type":"product","product_id":19,"product_name":"KitKat Chunky 50g","product_quantity":"50g","product_brand":"Nestle","product_price":2,"product_category":"Confectionery"},{"search_type":"product","product_id":5,"product_name":"Coca-Cola Cherry 1.5L","product_quantity":"1.5L","product_brand":"Coca-Cola","product_price":2.2,"product_category":"Beverages"}]!', 'name': 'get_products'}
]
prompt_response = '''ChatCompletion(id='chatcmpl-AB03Aq0x8drkP5uAvUSsA9IHDGPqT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "message": "I found some chocolate products for you:<br/><ul><li>KitKat White Chocolate 41.5g</li><li>Nestle Milkybar White Chocolate 25g</li><li>KitKat Chunky 50g</li><li>KitKat 4 Finger Bar 41.5g</li></ul>",\n  "product_ids": [27, 30, 19, 13]\n}', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1727185164, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=95, prompt_tokens=1125, total_tokens=1220, completion_tokens_details={'reasoning_tokens': 0}))'''

# Run the function
output = extract_product_ids(messages, prompt_response)
print(json.dumps(output, indent=2))


ValueError: No valid JSON found in prompt_response